In [ ]:
!pip install requests beautifulsoup4 pandas

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

In [4]:
import requests
from bs4 import BeautifulSoup

def get_guardian_articles_full(url, max_articles=10):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    articles = []
    links = soup.select("a[data-link-name='article']")
    seen = set()

    for link in links:
        href = link.get("href")
        title = link.get_text(strip=True)

        if href and href.startswith("http") and href not in seen and title:
            seen.add(href)
            article = {
                'title': title,
                'url': href
            }

            try:
                article_page = requests.get(href)
                article_soup = BeautifulSoup(article_page.text, 'html.parser')

                # Extract full article content (first 2-3 paragraphs as summary)
                paragraphs = article_soup.select("div[data-gu-name='body'] p")
                if not paragraphs:
                    paragraphs = article_soup.select("article p")

                description = " ".join(p.get_text(strip=True) for p in paragraphs[:3])
                article["description"] = description if description else "No description"


                # Try multiple ways to get date
                date_tag = article_soup.find("time")
                if date_tag and "datetime" in date_tag.attrs:
                    article["date"] = date_tag["datetime"]
                elif date_tag and date_tag.get_text(strip=True):
                    article["date"] = date_tag.get_text(strip=True)
                else:
                    # Try meta tag fallback
                    meta_date = article_soup.find("meta", {"property": "article:published_time"})
                    if meta_date and "content" in meta_date.attrs:
                        article["date"] = meta_date["content"]
                    else:
                        article["date"] = "No date"



            except Exception as e:
                article["description"] = "Description fetch error"
                article["date"] = "No date"

            articles.append(article)

        if len(articles) >= max_articles:
            break

    return articles


In [5]:
BASE_URL = "https://www.theguardian.com/international"
data = get_guardian_articles_full(BASE_URL, max_articles=10)

import pandas as pd
df = pd.DataFrame(data)
df.to_csv("guardian_articles_updated.csv", index=False)

print("Total articles scraped:", len(df))
df.head()


Total articles scraped: 10


,title,url,description,date
0,Self-belief and sex eggs: 10 things we learned...,https://www.theguardian.com/film/2025/jul/30/1...,When the author Amy Odell approached Gwyneth P...,2025-07-30T15:14:18.000Z
1,‘Everybody was fondling underwater!’: the Rock...,https://www.theguardian.com/film/2025/jul/31/t...,The Rocky Horror Picture Show was released in ...,2025-07-31T04:00:38.000Z
2,This trade deal is the EU’s Suez moment – its ...,https://www.theguardian.com/commentisfree/2025...,The Suez crisis in 1956 was a humiliating mome...,2025-07-31T07:00:41.000Z
3,Don’t call it morning sickness: ‘At times in m...,https://www.theguardian.com/lifeandstyle/2025/...,"The year my body revolted, I read all 1,296 pa...",2025-07-31T04:00:39.000Z
4,We are witnessing the silencing of American media,https://www.theguardian.com/commentisfree/2025...,The latest casualty ofDonald Trump’sefforts to...,2025-07-30T12:00:02.000Z


In [7]:
df.to_csv("guardian_articles.csv", index=False)


In [8]:
from google.colab import files
files.download("guardian_articles.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>